## Лекция 2  BM5    

## Функция ранжирования bm25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{TF(q_i,D)*(k+1)}{TF(q_i,D)+k(1-b+b\frac{l(d)}{avgdl})} $$ 
где   
>$TF(q_i,D)$ - частота слова $q_i$ в документе $D$      
$l(d)$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k$ и $b$ — свободные коэффициенты, обычно их выбирают как $k$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ - это модернизированная версия IDF: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [0]:
!pip install pymorphy2

In [0]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import pymorphy2 as pm
import string
import re

import csv

In [0]:
def get_data(file):
  '''Функция из файла делает списки запросов'''
  with open(file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader, None)
        documents_1 = []
        documents_2 = []
        data = []
        for row in reader:
          documents_1.append(row[1])
          documents_2.append(row[2])
          data.append([int(row[0])])
            
  return data[:1000], documents_1[:1000], documents_2[:1000]

data, documents_1, documents_2= get_data('quora_question_pairs_rus.csv')

In [0]:
#import nltk
#nltk.download('stopwords')
stopWords = list(stopwords.words('russian'))
morph = pm.MorphAnalyzer()

def preprocess(text : list) -> list:
    """Функция на вход получает текст и возвращает список нормализованных слов, без знаков пунктуации, без стопслов"""
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    filtered_words = list(filter(lambda token: token not in stopwords.words('russian'), tokens))
    norm_words = [morph.parse(token)[0].normal_form for token in filtered_words]
    return norm_words

In [44]:
def avg_length(documents : list) -> float:
  '''Считает среднюю длину документа в коллекции в словах'''  
  avg_length = sum(np.array([len(document.split()) for document in documents])) / len(documents)
  return float(avg_length)
print(avg_length(documents_1), avg_length(documents_2))

9.22558 9.42693


In [0]:
# 1) Dictionary doc : index and reverse index : doc
# 2) Dictionary index_doc : len(doc) 
index2doc = dict(enumerate(documents_1))
doc2index = {value : key for key, value in index2doc.items()}
doc2len = {key : len(value.split()) for key, value in index2doc.items()}
#index2doc, doc2index, doc2len

In [0]:
#Теперь сделаем predprocess всех текстов и создадим словарь idx2norm_doc
docs_processed = [preprocess(document) for document in documents_1]
idx2doc_n = dict(enumerate(docs_processed))
#idx2doc_n


In [0]:
N = len(documents_1)
avg_length = avg_length(documents_1)

In [0]:
#from collections import defaultdict
#Считаем TF 
def reverse_indx(idx2doc_n : dict):
    """
    Функция на вход получает индексированный словарь, на выход - обратный индекс
    Где для каждого токена есть словарь вида index_doc : tf
    """
    #ind_doc2tf = defaultdict(lambda: int())
    ind_doc2tf={}
    for key, value in idx2doc_n.items():
        for item in value:
            tf = value.count(item)
            if item not in ind_doc2tf.keys():
                #reverse_index_dict[item] = {tf : key}
                ind_doc2tf[item] = {key:tf}
            else:
                #reverse_index_dict[item][tf] = key
                ind_doc2tf[item][key]  = tf
    return ind_doc2tf

reverse_d = reverse_indx(idx2doc_n)
#reverse_d

In [0]:
#Сколько докумментов содержат слово. 
#word2count_doc = defaultdict(lambda: int())
word2count_doc = {key : len(value.keys()) for key, value in reverse_d.items()}
#word2count_doc

In [0]:
### реализуйте эту функцию ранжирования 
from math import log

def bm25(document, query, k=2.0, b=0.75,
         N=N, avg_length=avg_length, reverse_d=reverse_d, 
         index2doc=index2doc, doc2index=doc2index, word2count_doc=word2count_doc) -> float:
    '''Функция ранжирования bm25 для одного запроса для документа'''
        
    try:
      doc_id = doc2index[document] 
    except:
      return 0.0
    #document_processed = preprocess(document)
    document_processed = idx2doc_n[doc_id]
    query = preprocess(query)
    #print(document_processed, query)
  
    
    bm_total = 0
    for word in query:
      
      tf  = reverse_d.get(word,{-1:0}).get(doc_id,0)
      #print(tf)
      count_docs= word2count_doc.get(word, 0)
      #print(count_docs)
      
      idf = log((N-count_docs +0.5)/(count_docs + 0.5))
      bm = idf * (tf * (k+1) / (tf + k * (1-b+b*(len(document_processed)/avg_length))))
      bm_total += bm
    
    return bm_total 

### __Задача 1__:    
Напишите два поисковика на *BM25*. Один через подсчет метрики по формуле для каждой пары слово-документ, второй через умножение матрицы на вектор. 

Сравните время работы поиска на 100к запросах. В качестве корпуса возьмем 
[Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian).

In [0]:
import time
from collections import OrderedDict

In [34]:
len(documents_2), len(documents_1)

(1000, 1000)

In [35]:
#Первый вариант, считаем метрику по формуле для каждой пары слово документ. 
#Пусть запрос - это documents_2, а документ - documents_1
%%time
for i, query in enumerate(documents_2[:1000]):  
  if i % 10 == 0:
    print(i // 10, 'out of', 100)
  bms = [(0,0)]
  for document in documents_1[:1000]:
    bm = bm25(document, query)        
    if bm > bms[0][0]:
      bms.insert(0, (bm, document))

  for bm in bms[:10]:
    pass
    #print(dict_bm25[bm] + ':' + str(bm))

0 out of 100
1 out of 100
2 out of 100
3 out of 100
4 out of 100
5 out of 100
6 out of 100
7 out of 100
8 out of 100
9 out of 100
10 out of 100
11 out of 100
12 out of 100
13 out of 100
14 out of 100
15 out of 100
16 out of 100
17 out of 100
18 out of 100
19 out of 100
20 out of 100
21 out of 100
22 out of 100
23 out of 100
24 out of 100
25 out of 100
26 out of 100
27 out of 100
28 out of 100
29 out of 100
30 out of 100
31 out of 100
32 out of 100
33 out of 100
34 out of 100
35 out of 100
36 out of 100
37 out of 100
38 out of 100
39 out of 100
40 out of 100
41 out of 100
42 out of 100
43 out of 100
44 out of 100
45 out of 100
46 out of 100
47 out of 100
48 out of 100
49 out of 100
50 out of 100
51 out of 100
52 out of 100
53 out of 100
54 out of 100
55 out of 100
56 out of 100
57 out of 100
58 out of 100
59 out of 100
60 out of 100
61 out of 100
62 out of 100
63 out of 100
64 out of 100
65 out of 100
66 out of 100
67 out of 100
68 out of 100
69 out of 100
70 out of 100
71 out of 100
72

Поиск выполняется за 22.8 секунды для 100 запросов. 

### __Задача 2__:    



Выведите 10 первых результатов и их близость по метрике BM25 по запросу **рождественские каникулы** на нашем корпусе  Quora question pairs. 

In [0]:
def get_data(file):
  '''Функция из файла делает списки запросов'''
  with open(file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        next(reader, None)
        documents_1 = []
        documents_2 = []
        data = []
        for row in reader:
          documents_1.append(row[1])
          documents_2.append(row[2])
          data.append([int(row[0])])
            
  return data[:100000], documents_1[:100000], documents_2[:100000]

data, documents_1, documents_2= get_data('quora_question_pairs_rus.csv')

In [56]:
print(len(list(index2doc.keys())))

100000


In [60]:
bms = [(0,0)]
query = 'рождественские каникулы'
for document in documents_1:
  bm = bm25(document, query)
  if bm > bms[0][0]:
    bms.insert(0, (bm, document))
print(bms[:10])


[(14.998482869694746, 'какие рождественские традиции вы и ваша семья имеете на рождественский вечер и рождественский день'), (13.97776761709508, 'как вы проводите летние каникулы'), (12.892298236264834, 'какой лучший рождественский подарок для вас'), (9.920497087715205, 'которые являются лучшими местами для посещения в гоа во время каникул'), (9.714033240803387, 'что в вашем рождественском списке в этом году, на что вы надеетесь, анта принесет вам'), (9.376179865014187, 'что лучше всего безопасное место для 2 - 3-дневных каникул с моим gf в сентябре'), (0, 0)]


### __Задача 3__:    

Посчитайте точность поиска при 
1. BM25, b=0.75 
2. BM15, b=0 
3. BM11, b=1

In [69]:
bms = [(0,0)]
metric = 0
query = 'рождественские каникулы'
for document in documents_1[:1000]:
  bm = bm25(document, query, b=0.75)
  if bm > bms[0][0]:
    bms.insert(0, (bm, document))
  d_set = set(preprocess(document))
  q_set = set(preprocess(query))
  metric += len(d_set & q_set) / len(d_set | q_set)
metric
print(metric)

0.0


In [0]:
bms = [(0,0)]
metric = 0
query = 'рождественские каникулы'
for document in documents_1:
  bm = bm25(document, query, b=0)
  if bm > bms[0][0]:
    bms.insert(0, (bm, document))
  d_set = set(preprocess(document))
  q_set = set(preprocess(query))
  metric += len(d_set & q_set) / len(d_set | q_set)
  metric = metric / 5
print(metric)

In [0]:
bms = [(0,0)]
metric = 0
query = 'рождественские каникулы'
for document in documents_1:
  bm = bm25(document, query, b=1)
  if bm > bms[0][0]:
    bms.insert(0, (bm, document))
  d_set = set(preprocess(document))
  q_set = set(preprocess(query))
  metric += len(d_set & q_set) / len(d_set | q_set)
  metric = metric / 5
print(metric)